In [167]:
import sqlalchemy
import numpy as np
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from datetime import datetime
from sqlalchemy import Column, text, create_engine
from sqlalchemy.types import TIMESTAMP
from sqlalchemy.dialects.mysql import BIGINT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import glob

In [168]:
path="C:\\Users\\makol\\Documents\\Python_Scripts\\Git\\user_scripts\\masha_k\\Analyze_validation\\ngpt\\"

In [169]:
df=pd.DataFrame()

In [170]:
for i,f in enumerate(glob.iglob(path+'2020.08*.csv')):
    df_prom=(pd.read_csv(f,encoding='windows-1251',sep=','))
    df_prom['c']=1
    df_prom2=pd.DataFrame(df_prom.groupby(['Код Маршрута']).agg({'c': np.sum})).reset_index()
    df=df.append(df_prom)

C:\Users\makol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: Unable to allocate 99.1 MiB for an array with shape (4, 3248815) and data type object

In [14]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://makolesnikova@infraeconomy.com:zahceelaS4aewie7ouyohjoFuphah9Gu@84.201.144.84:54327/cei_samara')

In [15]:
meta = MetaData()
sti_val = Table('reestr_sti_validation', meta, autoload=True, autoload_with=engine)
conn = engine.connect()
Session = sessionmaker(bind = engine)
session = Session()

In [89]:
reestr_sti_validation=pd.read_sql_table('reestr_sti_validation', conn) 
reestr_sti_delete=pd.read_sql_table('reestr_sti_delete', conn) 

In [165]:
validations_validationngpt2019=pd.read_sql('SELECT * from validations_validationngpt  where date_part(\'month\',trip_datetime)=10 and date_part(\'year\',trip_datetime)=2019', conn)

In [ ]:
validations_validationngp2019['potok']=1

In [12]:
val2019.DataFrame(validations_validationngp2019.groupby(['val_marsh_id']).agg({'potok': np.sum})).reset_index()

,id,trip_datetime,terminal_id,ticket_num,crystal_id,num_trip,tabel_num,carrier_id,metro_station_id,ticket_type_id,val_marsh_id
0,10136066,2019-11-24 11:14:36+00:00,81129628,18,4DFA57C1,0,995,None,None,1,12203456789


In [ ]:
val2019['val_id_marsh2019']=val_marsh_id['val_marsh_id2019']

In [90]:
yandex=pd.read_sql_table('reestr_sti_yandexstimapping', conn)

In [152]:
reestr_sti_routesti=pd.read_sql_table('reestr_sti_routesti', conn,columns=[
    'id','city','route_type','route_number','route_name','working_hours','all_intervals',
'interval_min_workdays','interval_max_workdays', 
'interval_min_weekends','interval_max_weekends',
'interval_min_evening','interval_max_evening','evening_start_time','all_amounts_ts',
'all_route_times', 'route_time', 'all_route_lengths', 'route_lengths',
'company_name', 'model_ts','type_ts_id'])

In [154]:
sql ="SELECT geometry as geom, id, direction, route_trip_id  FROM public.routes_routetripgeometry"
routes_routetripgeometry=gpd.read_postgis(sql, conn)

In [155]:
routes_routetripgeometry.head()

,geom,id,direction,route_trip_id
0,"MULTILINESTRING ((50.18854 53.19159, 50.18817 ...",1757,0,2193179787
1,"MULTILINESTRING ((50.08948 53.18230, 50.08981 ...",1758,1,2193180144
2,"MULTILINESTRING ((50.30940 53.31019, 50.30877 ...",1759,0,2193179805
3,"MULTILINESTRING ((50.07599 53.18359, 50.07607 ...",1760,1,2193179806
4,"MULTILINESTRING ((50.08948 53.18230, 50.08981 ...",1761,0,2193179898


In [156]:
reestr_sti_routesti['sti_route_id']=reestr_sti_routesti['id']

In [157]:
len(reestr_sti_routesti)

6249

In [158]:
reestr_sti_routesti=reestr_sti_routesti.merge(reestr_sti_delete,on='sti_route_id',how='left')

In [159]:
reestr_sti_routesti=reestr_sti_routesti[reestr_sti_routesti['is_delete']!=1]

In [160]:
yandex.head()

,id,sti_route_id,yandex_route_id
0,1,726,1704841444
1,2,3063,3245401927
2,3,3064,3245401927
3,4,3065,3245401927
4,5,3066,3245401927


In [164]:
reestr_sti_routesti=reestr_sti_routesti.merge(reestr_sti_validation,on='sti_route_id',how='left')

In [162]:
reestr_sti_routesti=reestr_sti_routesti.merge(yandex,on='sti_route_id',how='left')

In [166]:
reestr_sti_routesti=reestr_sti_routesti.drop(columns=['id_x','id_y']).drop_duplicates()

KeyError: "['id_x' 'id_y'] not found in axis"

In [ ]:
reestr_sti_routesti=reestr_sti_routesti.merge(val2019,on='val_id_marsh2019,how='left')

In [151]:
len(reestr_sti_routesti)

0

In [ ]:
reestr_sti_routesti=reestr_sti_routesti.merge(yandex,on='sti_route_id',how='left')